In [1]:
using LinearAlgebra, ForwardDiff #for norm and differentiation

## Demo 1

In [13]:
function Newt_Raph(f,ϵ,x0)
    n = 0 # iteration count
    ∇(f,x) = ForwardDiff.jacobian(f,x)
    d = inv(-∇(f,x0)) * f(x0)
    while norm(d) > ϵ
        x1 = x0 + d
        x0 = x1
        d = inv(-∇(f,x0)) * f(x0)
        n = n+1
        println("Iteration $(n): $x0")
    end
    println("After $(n) iterations the solution is x = $(x0)")
end

Newt_Raph (generic function with 1 method)

In [17]:
# define intial point 
x0 = [1, 1, 1]

# define ϵ
ϵ = 0.001

# create an array of function (make sure there is the same number of arguments in each row/column)
f(x) = [x[1]^2 + x[2]^2 - 4*x[3] - 0;
        x[1]^2 + x[3]^2 + 0*x[3] - (1/4);
        x[1]^2 + x[2]^2 + x[3]^2 - 1]


f (generic function with 1 method)

In [18]:
Newt_Raph(f, ϵ, x0)

Iteration 1: [0.7916666666666665, 0.875, 0.33333333333333337]
Iteration 2: [0.5236528822055138, 0.8660714285714286, 0.23809523809523808]
Iteration 3: [0.4473267879691847, 0.8660254050073638, 0.23606889564336372]
Iteration 4: [0.4408110310419283, 0.8660254037844386, 0.23606797749997818]
After 4 iterations the solution is x = [0.4408110310419283, 0.8660254037844386, 0.23606797749997818]


## Demo 2

In [15]:
function newton_syst(A, X, ρ, β)
    ## Calculate parmeters
    (m,n) = size(A)
    e = ones(n)
    
    ## Diagonalize z
    z = ρ*inv(X)*e
    Z = Diagonal(z)


    ## Newton direction (our left hand side matrix)
    Nmatrix = [zeros(n,n) A' Diagonal(ones(n));
                A zeros(m,m) zeros(m,n);
                Z zeros(n,m) X]

    ## Calculate our right hand side vector components (residual dual, primal, and center)
    rd = zeros(n)
    rp = zeros(m)
    rc = X*Z*e - (ρ*β)*e

    ## Define right hand side vector
    Nrhs = -[rd; rp; rc]

    ## Solve for Δ vector (use backslash operator)
    Δ = Nmatrix \ Nrhs

    Δx = Δ[1:n]
    Δμ = Δ[n+1:n+m]
    Δz = Δ[n+m+1:end]
    println("Δx = $(Δx),\nΔμ = $(Δμ),\nΔz = $(Δz)")
    
end

newton_syst (generic function with 1 method)

In [3]:
# define inputs for function: need A, X, ρ, β
A = [3 1 1 0;
    0 1 0 1]
X = [2 0 0 0;
    0 4 0 0;
    0 0 8 0;
    0 0 0 2]
ρ = 10
β = 0.5

0.5

In [16]:
# Solve Newton System
newton_syst(A,X,ρ,β)

Δx = [-0.2325581395348837, 0.6046511627906976, 0.09302325581395224, -0.6046511627906976],
Δμ = [0.6395348837209303, 0.9883720930232558],
Δz = [-1.9186046511627908, -1.627906976744186, -0.63953488372093, -0.9883720930232559]


## Problem 1

In [113]:
x0 = [2 4]
f(x) = x[1]^4 + 2*x[2]^2
A = [1 2]
ϵ = 0.00001 

function newton_equality(f,A,x0,ϵ)
    H(f,x) = ForwardDiff.hessian(f,x)
    ∇(f,x) = ForwardDiff.gradient(f,x)
    n = 0
    norm_crit = 100.0
    while norm_crit > ϵ
        Newt_matrix = [H(f,x0) A';
                        A 0]
        Newt_rhs = [-∇(f,x0)'; 0] 
        d1 = Newt_matrix \ Newt_rhs
        x1 = x0 + d1[1:2]'
        norm_crit = norm(x1 -x0)
        x0 = x1
        n += 1
        println("Iteration $(n): $x0")
    end
    println("After $(n) iterations the solution is x = $(x0), \nOptimal objective function value is $(f(x0))")
end


newton_equality (generic function with 1 method)

In [114]:
newton_equality(f,A,x0,ϵ)


Iteration 1: [1.5102040816326532 4.244897959183674]
Iteration 2: [1.3238151367310684 4.3380924316344665]
Iteration 3: [1.296411946573941 4.351794026713031]
Iteration 4: [1.2958522924386668 4.352073853780668]
Iteration 5: [1.2958520620959688 4.352073968952017]
After 5 iterations the solution is x = [1.2958520620959688 4.352073968952017], 
Optimal objective function value is 40.70091767599013


In [102]:
using JuMP, Ipopt
m = Model(with_optimizer(Ipopt.Optimizer))

@variable(m, x[1:2] >= 0)
@constraint(m, x[1] + 2*x[2] == 10)
@NLobjective(m, Min, x[1]^4 + 2*x[2]^2)

optimize!(m)

x_value = value.(x)
println("$(x_value), $(objective_value(m)) ")


This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

## Problem 3

In [5]:
A3 = [2 1 1; 1 2 0]
X3 = [2 0 0; 0 4 0; 0 0 1]
ρ3 = 5
β3 = 0.5

0.5

In [17]:
# Solve Newton System
newton_syst(A3,X3,ρ3,β3)

Δx = [0.2236024844720495, -0.11180124223602483, -0.3354037267080746],
Δμ = [0.8229813664596272, -0.11645962732919239],
Δz = [-1.5295031055900619, -0.5900621118012424, -0.8229813664596272]
